# CS246 - Homework 1
## Question 2



### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive2
#the output 'xxx is not a symbolic link' will not affect your implementation or execution
#to fix 'xxx is not a symbolic link', you can uncomment the lines starting from !mv xxxx
#you may need to replace xxx.11 with the correct version if other errors come up after colab update
#to get the correct version, use !ls /usr/local/lib to find out
!mv /usr/local/lib/libtbbmalloc_proxy.so.2 /usr/local/lib/libtbbmalloc_proxy.so.2.backup
!mv /usr/local/lib/libtbbmalloc.so.2 /usr/local/lib/libtbbmalloc.so.2.backup
!mv /usr/local/lib/libtbbbind_2_5.so.3 /usr/local/lib/libtbbbind_2_5.so.3.backup
!mv /usr/local/lib/libtbb.so.12 /usr/local/lib/libtbb.so.12.backup
!mv /usr/local/lib/libtbbbind_2_0.so.3 /usr/local/lib/libtbbbind_2_0.so.3.backup
!mv /usr/local/lib/libtbbbind.so.3 /usr/local/lib/libtbbbind.so.3.backup
!ln -s /usr/local/lib/libtbbmalloc_proxy.so.2.11 /usr/local/lib/libtbbmalloc_proxy.so.2
!ln -s /usr/local/lib/libtbbmalloc.so.2.11 /usr/local/lib/libtbbmalloc.so.2
!ln -s /usr/local/lib/libtbbbind_2_5.so.3.11 /usr/local/lib/libtbbbind_2_5.so.3
!ln -s /usr/local/lib/libtbb.so.12.11 /usr/local/lib/libtbb.so.12
!ln -s /usr/local/lib/libtbbbind_2_0.so.3.11 /usr/local/lib/libtbbbind_2_0.so.3
!ln -s /usr/local/lib/libtbbbind.so.3.11 /usr/local/lib/libtbbbind.so.3
#If error related to the above execution occurs, you can try commenting out the above 12 lines under pip install PyDrive2 (not included)

# !sudo ldconfig
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=638cedd33e33237a7dc6d979ff3591d12486ac4bc2cba021fe64ecc8323684fe
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 1.5 MB/s eta 0:00:00
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 30 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 M

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1sA1pnEs5gWb-yjW-3RRfpn_cE0V2LXet'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('browsing.txt')

In [ ]:
# Let's import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.


In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [ ]:
spark

# Question 2

In [ ]:
txt2 = spark.read.text("browsing.txt")

In [ ]:
# Split the lines into lists of items
data = txt2.select(array_distinct(split(col("value"), " ")).alias("items"))\
          .withColumn("id", monotonically_increasing_id())

In [ ]:
df = data.select("id", slice("items", 1, size("items") - 1).alias("items"))

In [ ]:
df.show(truncate=False)

+---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |items                                                                                                                                                                                         |
+---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0  |[FRO11987, ELE17451, ELE89019, SNA90258, GRO99222]                                                                                                                                            |
|1  |[GRO99222, GRO12298, FRO12685, ELE91550, SNA11465, ELE26917, ELE52966, FRO90334, SNA30755, ELE17451, FRO84225, SNA80192]                                                                      |
|2  |[ELE17451,

In [ ]:
c = data.count()

In [ ]:
from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="items", minSupport = 100/c)
model = fpGrowth.fit(df)

In [ ]:
association_rules_2 = model.associationRules\
                           .filter(size("antecedent") == 1)\
                           .filter(size("consequent") == 1)\
                           .orderBy("antecedent", ascending=True)\
                           .orderBy("confidence", ascending=False)

In [ ]:
association_rules_2.take(5)

[Row(antecedent=['DAI93865'], consequent=['FRO40251'], confidence=1.0, lift=8.013656274156146, support=0.006687887849265297),
 Row(antecedent=['GRO85051'], consequent=['FRO40251'], confidence=0.999176276771005, lift=8.00705523933394, support=0.039001961351725026),
 Row(antecedent=['GRO38636'], consequent=['FRO40251'], confidence=0.9906542056074766, lift=7.938762290285528, support=0.0034082505385678915),
 Row(antecedent=['ELE12951'], consequent=['FRO40251'], confidence=0.9905660377358491, lift=7.93805574326788, support=0.00337609723160027),
 Row(antecedent=['DAI88079'], consequent=['FRO40251'], confidence=0.9867256637168141, lift=7.907280305915135, support=0.014340374907559243)]

In [ ]:
def smaller_lexicographic_element(arr):
    return sorted(arr)[0] if arr else None

association_rules_3 = model.associationRules\
                           .filter(size("antecedent") == 2)\
                           .filter(size("consequent") == 1)\
                           .orderBy("consequent", ascending=True)\
                           .orderBy(col("antecedent").getItem(1), ascending=True)\
                           .orderBy(col("antecedent").getItem(0), ascending=True)\
                           .orderBy("confidence", ascending=False)

In [ ]:
association_rules_3.show()

+--------------------+----------+------------------+-----------------+--------------------+
|          antecedent|consequent|        confidence|             lift|             support|
+--------------------+----------+------------------+-----------------+--------------------+
|[ELE20847, GRO85051]|[FRO40251]|               1.0|8.013656274156146|0.004469309668499...|
|[GRO85051, GRO38814]|[FRO40251]|               1.0|8.013656274156146|0.003697630301276...|
|[DAI23334, ELE92920]|[DAI62779]|               1.0|4.664916754162292|0.004597922896369...|
|[GRO85051, FRO53271]|[FRO40251]|               1.0|8.013656274156146| 0.00337609723160027|
|[GRO85051, GRO21487]|[FRO40251]|               1.0|8.013656274156146|0.003858396836114...|
|[GRO85051, SNA80324]|[FRO40251]|               1.0|8.013656274156146|0.015144207581749784|
|[GRO85051, GRO73461]|[FRO40251]|               1.0|8.013656274156146|0.004726536124240378|
|[GRO85051, DAI31081]|[FRO40251]|               1.0|8.013656274156146|0.00327963